## CLIF Table One

Author: Kaveri Chhikara
Date v1: May 13, 2025

This script identifies the cohort of encounters with at least one ICU stay and then summarizes the cohort data into one table. 


#### Requirements

* Required table filenames should be `clif_patient`, `clif_hospitalization`, `clif_adt`, `clif_vitals`, `clif_labs`, `clif_medication_admin_continuous`, `clif_respiratory_support`, `clif_patient_assessments`
* Within each table, the following variables and categories are required.

| Table Name | Required Variables | Required Categories |
| --- | --- | --- |
| **patient** | `patient_id`, `race_category`, `ethnicity_category`, `sex_category`, `death_dttm` | - |
| **hospitalization** | `patient_id`, `hospitalization_id`, `admission_dttm`, `discharge_dttm`,`discharge_dttm`, `age_at_admission` | - |
| **adt** |  `hospitalization_id`, `hospital_id`,`in_dttm`, `out_dttm`, `location_category` | - |
| **vitals** | `hospitalization_id`, `recorded_dttm`, `vital_category`, `vital_value` | weight_kg |
| **labs** | `hospitalization_id`, `lab_result_dttm`, `lab_order_dttm`, `lab_category`, `lab_value_numeric` | creatinine, bilirubin_total, po2_arterial, platelet_count |
| **medication_admin_continuous** | `hospitalization_id`, `admin_dttm`, `med_name`, `med_category`, `med_dose`, `med_dose_unit` | norepinephrine, epinephrine, phenylephrine, vasopressin, dopamine, angiotensin(optional) |
| **respiratory_support** | `hospitalization_id`, `recorded_dttm`, `device_category`, `mode_category`,  `fio2_set`, `lpm_set`, `resp_rate_set`, `peep_set`, `resp_rate_obs`, `tidal_volume_set`, `pressure_control_set`, `pressure_support_set` | - |
| **patient_assessments** | `hospitalization_id`, `recorded_dttm` , `assessment_category`, `numerical_value`| `gcs_total` |
| **crrt_therapy** | `hospitalization_id`, `recorded_dttm` | - |

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import pyCLIF
import pyarrow
import sofa_score
from datetime import timedelta

## import outlier json
with open('../config/outlier_config.json', 'r') as f:
    outlier_cfg = json.load(f)



## Load data

In [ ]:
patient = pyCLIF.load_data('clif_patient')
hospitalization = pyCLIF.load_data('clif_hospitalization')
adt = pyCLIF.load_data('clif_adt')

# ensure id variable is of dtype character
hospitalization['hospitalization_id']= hospitalization['hospitalization_id'].astype(str)
patient['patient_id']= patient['patient_id'].astype(str)
adt['hospitalization_id']= adt['hospitalization_id'].astype(str)

# check for duplicates
# patient table should be unique by patient id
patient = pyCLIF.remove_duplicates(patient, ['patient_id'], 'patient')
# hospitalization table should be unique by hospitalization id
hospitalization = pyCLIF.remove_duplicates(hospitalization, ['hospitalization_id'], 'hospitalization')
# adt table should be unique by hospitalization id and in dttm
adt = pyCLIF.remove_duplicates(adt, ['hospitalization_id', 'hospital_id', 'in_dttm'], 'adt')

hospitalization = hospitalization.sort_values(['hospitalization_id', "admission_dttm"])
adt = adt.sort_values(['hospitalization_id', "in_dttm"])

In [ ]:
# Standardize all _dttm variables to the same format
patient = pyCLIF.convert_datetime_columns_to_site_tz(patient,  pyCLIF.helper['timezone'])
hospitalization = pyCLIF.convert_datetime_columns_to_site_tz(hospitalization, pyCLIF.helper['timezone'])
adt = pyCLIF.convert_datetime_columns_to_site_tz(adt,  pyCLIF.helper['timezone'])

## Cohort

Filter down to adult encounters with at least one recorded ICU stay.

In [4]:
adult_hosp_ids = hospitalization[(hospitalization['admission_dttm'].dt.year <= 2024) & 
                                (hospitalization['age_at_admission'] >= 18)]['hospitalization_id'].unique()
icu_hosp_ids = adt[adt['location_category'].str.lower() == 'icu']['hospitalization_id'].unique()
# Get intersection of adult and ICU hospitalizations
cohort_ids = np.intersect1d(adult_hosp_ids, icu_hosp_ids)

## Linked Hospitalizations

If the `id_col` supplied by user is `hospitalization_id`, then we combine multiple `hospitalization_ids` into a single `encounter_block` for patients who transfer between hospital campuses or return soon after discharge. Hospitalizations that have a gap of **6 hours or less** between the discharge dttm and admission dttm are put in one encounter block.

If the `id_col` supplied by user is `hospitalization_joined_id` from the hospitalization table, then we consider the user has already stitched similar encounters, and we will consider that as the primary id column for all table joins moving forward.

In [5]:
hospitalization_cohort = hospitalization[hospitalization['hospitalization_id'].isin(cohort_ids)].copy()
adt_cohort = adt[adt['hospitalization_id'].isin(cohort_ids)].copy()

In [ ]:
if pyCLIF.helper['id_column'] == 'hospitalization_id':
    stitched_cohort = pyCLIF.stitch_encounters(hospitalization_cohort, adt_cohort, time_interval=6)
    id_col = 'encounter_block'
    stitched_unique = stitched_cohort[['patient_id', id_col]].drop_duplicates()
    all_ids = stitched_cohort[['patient_id', 'hospitalization_id', id_col, 'discharge_category', 'admission_dttm', 'discharge_dttm', 'age_at_admission']].drop_duplicates()
else:
    id_col = pyCLIF.helper['id_column']
    all_ids = hospitalization[['patient_id', 'hospitalization_id', id_col, 'discharge_category','admission_dttm', 'discharge_dttm', 'age_at_admission']].drop_duplicates()

In [ ]:
# Get start and End Years
start_year = all_ids['admission_dttm'].dt.year.min()
end_year = all_ids['admission_dttm'].dt.year.max()
all_ids['is_dead'] = (all_ids['discharge_category'].str.lower().isin(['expired', 'hospice'])).astype(int)
print("Start Year:", start_year, "\n End Year:", end_year)

## Demographics

Summarize the demographic info at patient level

In [8]:
# Get patient demographics for patients in our cohort
patient_cohort = patient[patient['patient_id'].isin(all_ids['patient_id'])].copy()

In [9]:
patient_cohort =pyCLIF.map_race_column(patient_cohort, 'race_category')
patient_cohort['race_new'] = patient_cohort['race_new'].fillna('Missing')
patient_cohort['ethnicity_category'] = patient_cohort['ethnicity_category'].fillna('Missing')
patient_cohort['sex_category'] = patient_cohort['sex_category'].fillna('Missing')

In [ ]:
patient_cohort.columns

## Hospital and ICU Admission Summary

1. Get the first ICU dttm. 
2. Calculate ICU LOS and Hospital LOS for each encounter in days. 

In [11]:
hosp_admission_summary = (
        adt_cohort
        .merge(all_ids[['hospitalization_id', id_col]], on='hospitalization_id')
        .groupby(id_col)
        .agg(
            min_in_dttm = ('in_dttm', 'min'),
            max_out_dttm = ('out_dttm', 'max'),
            first_admission_location = ('location_category', 'first')
        )
)
hosp_admission_summary['hospital_length_of_stay_days'] = (
    (hosp_admission_summary['max_out_dttm'] - hosp_admission_summary['min_in_dttm']) / pd.Timedelta(days=1))

In [12]:
# lowercase the column, not the entire df
adt_cohort['location_category'] = (
    adt_cohort['location_category']
    .str.lower()
)

# Merge on the ID mapping
df = (
    adt_cohort
    .merge(all_ids[['hospitalization_id', id_col]], on='hospitalization_id')
)

# restrict to ICU rows
icu_df = df.query('location_category == "icu"')

# find first ICU in time per id_col
first_in = (
    icu_df
     .groupby(id_col, as_index=False)
     .agg(first_icu_in_dttm=('in_dttm', 'min'))
)

# join back to pull the matching out_dttm
icu_summary = (
    first_in
      # bring in that one row’s out_dttm
      .merge(
          icu_df[['hospitalization_id','in_dttm','out_dttm', id_col]],
          left_on=[id_col, 'first_icu_in_dttm'],
          right_on=[id_col, 'in_dttm'],
          how='left'
      )
      .rename(columns={'out_dttm':'first_icu_out_dttm'})
)

# compute LOS in days (out - in)
icu_summary['first_icu_los_days'] = (
    (icu_summary['first_icu_out_dttm'] - icu_summary['first_icu_in_dttm'])
    .dt.total_seconds()
    / (3600 * 24)
)

# trim to just the columns you need
icu_summary = icu_summary[[id_col, 'first_icu_in_dttm',
                           'first_icu_out_dttm','first_icu_los_days']]

In [13]:
# Merge all_ids with icu_summary and hosp_admission_summary
final_df = (
    all_ids[[id_col,  'discharge_category', 'admission_dttm', 'discharge_dttm', 'age_at_admission', 'is_dead']]
    .merge(icu_summary, on=id_col, how='left')
    .merge(hosp_admission_summary, on=id_col, how='left')
)
final_df['first_admission_location'] = final_df['first_admission_location'].fillna('Missing')

## IMV Encounters

For the most reliable results, we should run the waterfall on the respiratory support table before summarising. 
To prioritize efficiency, we have not done so for this version of the code.

In [ ]:
rst_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'device_name',
    'device_category',
    'mode_name', 
    'mode_category',
    'tracheostomy',
    'fio2_set',
    'lpm_set',
    'resp_rate_set',
    'peep_set',
    'tidal_volume_set', 
    'pressure_control_set',
    'pressure_support_set',
]

resp_support_raw = pyCLIF.load_data(
    'clif_respiratory_support',
    columns=rst_required_columns,
    filters={'hospitalization_id': all_ids['hospitalization_id'].unique().tolist()}
)

resp_support = resp_support_raw.copy()
resp_support = pyCLIF.convert_datetime_columns_to_site_tz(resp_support, pyCLIF.helper['timezone'])
resp_support['device_category'] = resp_support['device_category'].str.lower()
resp_support['mode_category'] = resp_support['mode_category'].str.lower()
resp_support['lpm_set'] = pd.to_numeric(resp_support['lpm_set'], errors='coerce')
resp_support['resp_rate_set'] = pd.to_numeric(resp_support['resp_rate_set'], errors='coerce')
resp_support['peep_set'] = pd.to_numeric(resp_support['peep_set'], errors='coerce')
resp_support['tidal_volume_set'] = pd.to_numeric(resp_support['tidal_volume_set'], errors='coerce')
resp_support['pressure_control_set'] = pd.to_numeric(resp_support['pressure_control_set'], errors='coerce')
resp_support['pressure_support_set'] = pd.to_numeric(resp_support['pressure_support_set'], errors='coerce')

resp_support = resp_support.sort_values(['hospitalization_id', 'recorded_dttm'])
# del resp_support_raw

print("\n=== Apply outlier thresholds ===\n")
resp_support['fio2_set'] = pd.to_numeric(resp_support['fio2_set'], errors='coerce')
# (Optional) If FiO2 is >1 on average => scale by /100
fio2_mean = resp_support['fio2_set'].mean(skipna=True)
# If the mean is greater than 1, divide 'fio2_set' by 100
if fio2_mean and fio2_mean > 1.0:
    # Only divide values greater than 1 to avoid re-dividing already correct values
    resp_support.loc[resp_support['fio2_set'] > 1, 'fio2_set'] = \
        resp_support.loc[resp_support['fio2_set'] > 1, 'fio2_set'] / 100
    print("Updated fio2_set to be between 0.21 and 1")
else:
    print("FIO2_SET mean=", fio2_mean, "is within the required range")

resp_stitched = resp_support.merge(
    all_ids[['hospitalization_id', id_col]],
    on='hospitalization_id', how='right'
)

pyCLIF.apply_outlier_thresholds(resp_stitched, 'fio2_set', *outlier_cfg['fio2_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'peep_set', *outlier_cfg['peep_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'lpm_set',  *outlier_cfg['lpm_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'resp_rate_set', *outlier_cfg['resp_rate_set'])

#### Vent start and end times 

Calculate vent start times for the first episode of invasive mechanical intubation.   
Limitation: the vent end time might not be associated with the same intubation episode.

In [15]:
#  Identify IMV
imv_mask = resp_stitched['device_category'].str.contains("imv", case=False, na=False)
resp_stitched_imv = resp_stitched[imv_mask].copy()
# Create on_vent column for IMV records
resp_stitched_imv['on_vent'] = 1
# Get unique encounter IDs from resp_stitched_imv
imv_encounters = resp_stitched_imv[id_col].unique()
# Determine Vent Start/End for Each Hospitalization 
vent_start_end = resp_stitched_imv.groupby(id_col).agg(
    vent_start_time=('recorded_dttm','min'),
    vent_end_time=('recorded_dttm','max')
).reset_index()

In [16]:
# add on_vent flag to final_df
final_df = final_df.merge(
    resp_stitched_imv[[id_col, 'on_vent']].drop_duplicates(),
    on=id_col,
    how='left'
)
final_df['on_vent'] = final_df['on_vent'].fillna(0).astype(int)

### Initial Mode and Ventilator settings  

* Initial mode: First mode category post intubation
* Vent settings: Median and IQR for all non NA values.

In [17]:
# Get first non-NA mode category for each id_col
initial_modes = resp_stitched_imv.groupby(id_col)['mode_category'].first().reset_index()
initial_modes = initial_modes.rename(columns={'mode_category': 'initial_mode_category'})
initial_modes['initial_mode_category'] = initial_modes['initial_mode_category'].fillna('Missing')

# Merge back to final_df
final_df = final_df.merge(initial_modes, on=id_col, how='left')

In [18]:
# Filter resp_stitched to only those encounters and merge
resp_stitched_final = resp_stitched[resp_stitched[id_col].isin(imv_encounters)]

# 4) aggregate in one shot
numeric_cols = [
    'fio2_set','lpm_set','resp_rate_set','peep_set',
    'tidal_volume_set','pressure_control_set',
    'pressure_support_set'
]

# build named aggregation dict
named_aggs = {
    'mode_category': ('mode_category', 'first')
}
for col in numeric_cols:
    named_aggs[f'{col}_median'] = (col, 'median')
    named_aggs[f'{col}_q1']     = (col, lambda x: x.quantile(0.25))
    named_aggs[f'{col}_q3']     = (col, lambda x: x.quantile(0.75))

vent_stats = (
    resp_stitched_final
    .groupby(id_col, as_index=False)
    .agg(**named_aggs)
)

final_df = final_df.merge(vent_stats, on=id_col, how='left')

### First location of intubation

In [19]:
# Get minimal ADT cohort with required columns and merge with all_ids to get id_col
adt_cohort_subset = pd.merge(
    adt_cohort[['hospitalization_id', 'in_dttm', 'location_category']],
    all_ids[['hospitalization_id', id_col]],
    on='hospitalization_id'
)


adt_vent = pd.merge(
    vent_start_end[[id_col, 'vent_start_time']],
    adt_cohort_subset,
    on=id_col
)

adt_vent['time_diff'] = abs(adt_vent['vent_start_time'] - adt_vent['in_dttm'])

# Get the closest ADT row for each encounter block
closest_adt = (adt_vent
    .sort_values('time_diff')
    .groupby(id_col)
    .first()
    .reset_index()
)
closest_adt = closest_adt.rename(columns={'location_category': 'first_location_imv'})

final_df = final_df.merge(
    closest_adt[[id_col, 'first_location_imv']],
    on=id_col,
    how='left'
)

## Vitals- Weight

In [ ]:
vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]
vitals_of_interest = ['weight_kg']

vitals_cohort = pyCLIF.load_data('clif_vitals',
    columns=vitals_required_columns,
    filters={'hospitalization_id': all_ids['hospitalization_id'].unique().tolist(), 
             'vital_category': vitals_of_interest}
)
vitals_cohort = vitals_cohort.merge(all_ids[['hospitalization_id', id_col]], on='hospitalization_id', how='left')
vitals_cohort = pyCLIF.convert_datetime_columns_to_site_tz(vitals_cohort,  pyCLIF.helper['timezone'])
vitals_cohort['vital_value'] = pd.to_numeric(vitals_cohort['vital_value'], errors='coerce')
vitals_cohort = vitals_cohort.sort_values([id_col, 'recorded_dttm'])
is_weight = vitals_cohort['vital_category'] == 'weight_kg'
min_weight, max_weight = outlier_cfg['weight_kg']
vitals_cohort.loc[is_weight & (vitals_cohort['vital_value'] < min_weight), 'vital_value'] = np.nan
vitals_cohort.loc[is_weight & (vitals_cohort['vital_value'] > max_weight), 'vital_value'] = np.nan
vitals_cohort = vitals_cohort.dropna(subset=['vital_value'])

In [21]:
# Get average weight for each encounter block
weight_summary = (vitals_cohort
    .groupby(id_col)['vital_value']
    .mean()
    .reset_index()
    .rename(columns={'vital_value': 'weight_kg'})
)

## Vasopressors

In [ ]:
meds_of_interest = [
    'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
    'dopamine', 'angiotensin'
]

meds_required_columns = [
    'hospitalization_id',
    'admin_dttm',
    'med_name',
    'med_category',
    'med_dose',
    'med_dose_unit'
]

meds_filters = {
    'hospitalization_id': all_ids['hospitalization_id'].unique().tolist(),
    'med_category': meds_of_interest
}
meds = pyCLIF.load_data('clif_medication_admin_continuous', columns=meds_required_columns, filters=meds_filters)
# ensure correct format
meds['hospitalization_id']= meds['hospitalization_id'].astype(str)
meds['med_dose_unit'] = meds['med_dose_unit'].str.lower()
meds = pyCLIF.convert_datetime_columns_to_site_tz(meds,  pyCLIF.helper['timezone'])
meds['med_dose'] = pd.to_numeric(meds['med_dose'], errors='coerce')

In [23]:
meds_stitched = meds.merge(all_ids[['hospitalization_id', id_col]], on='hospitalization_id', how='left')

In [24]:
meds_filtered = meds_stitched[~meds_stitched['med_dose'].isnull()].copy()
# Standardize time units in med_dose_unit
meds_filtered['med_dose_unit'] = (meds_filtered['med_dose_unit']
    .str.replace('hour', 'hr')
    .str.replace('minutes', 'min')
    .str.replace('minute', 'min')
    .str.replace('hours', 'hr')
)
meds_filtered = meds_filtered[meds_filtered['med_dose_unit'].apply(pyCLIF.has_per_hour_or_min)].copy()
ne_df = meds_filtered.merge(weight_summary[[id_col, 'weight_kg']], on=id_col, how='left')
ne_df["med_dose_converted"] = ne_df.apply(pyCLIF.convert_dose, axis=1)
ne_df = ne_df[ne_df.apply(pyCLIF.is_dose_within_range, axis=1, args=(outlier_cfg,))].copy()

In [25]:
# Create flags for number of unique vasopressors received
vaso_flags = pd.get_dummies(
    ne_df.groupby(id_col)['med_category']
    .nunique()
    .reset_index()
    .rename(columns={'med_category': 'vaso_count'})
    .set_index(id_col)['vaso_count']
    .apply(lambda x: f'vasopressors_{x}')
).astype(int)
# Create flag for if patient was ever on any pressor
on_pressor = pd.DataFrame(
    ne_df.groupby(id_col).size() > 0,
    columns=['on_pressor']
).astype(int)

# Create flag for number of concurrent pressors at any time
n_pressors = pd.DataFrame(
    ne_df.groupby([id_col, 'admin_dttm'])['med_category']
    .nunique()
    .groupby(id_col)
    .max()
    .rename('n_pressors')
)

# Combine with existing vaso_flags
vaso_flags = pd.concat([vaso_flags, on_pressor, n_pressors], axis=1)

In [26]:
# Reset index to make sure id_col is a column, not an index
vaso_flags = vaso_flags.reset_index()
final_df = final_df.merge(
    vaso_flags[[id_col, 'on_pressor', 'n_pressors']],
    on=id_col,
    how='left'
)

In [27]:
dose_stats = (
    ne_df
    .groupby([id_col, 'med_category'], as_index=False)['med_dose_converted']
    .agg(
        median = 'median',
        q1     = lambda x: x.quantile(0.25),
        q3     = lambda x: x.quantile(0.75)
    )
)

# 2) Pivot into a wide format with MultiIndex columns
dose_wide = dose_stats.pivot(
    index=id_col,
    columns='med_category',
    values=['median','q1','q3']
)

# 3) Flatten the MultiIndex to single‐level: "<med>_<stat>"
dose_wide.columns = [
    f"{med.lower()}_{stat}"  # e.g. "norepinephrine_median", "norepinephrine_q1", ...
    for stat, med in dose_wide.columns
]
dose_wide = dose_wide.reset_index()

final_df = final_df.merge(
    dose_wide,
    on=id_col,
    how='left'
)

## SOFA Calculation

In [ ]:
tables_path= pyCLIF.helper['tables_path']
sofa_input_df = icu_summary[[id_col, 'first_icu_in_dttm']].copy()
sofa_input_df = sofa_input_df.rename(columns={'first_icu_in_dttm': 'start_dttm'})
sofa_input_df['stop_dttm'] = sofa_input_df['start_dttm'] + pd.Timedelta(hours=24)
id_mappings = all_ids[[id_col, 'hospitalization_id' ]].drop_duplicates()

sofa_df = sofa_score.compute_sofa(
            ids_w_dttm = sofa_input_df,          # id, start_dttm, end_dttm  (local time)
            tables_path = tables_path,
            use_hospitalization_id = False,         # or False + id_mapping (new id , hospitalization_id)
            id_mapping = id_mappings,              # first column should be your new id_variable, second column is hospitalization id
            helper_module = pyCLIF,               
            output_filepath = "../output/intermediate/sofa.parquet"
         )

In [29]:
final_df = (
    final_df
    .merge(sofa_df, on=id_col, how='left')
)

In [30]:
final_df = (
    final_df
    .merge(all_ids[[id_col, 'patient_id']], on=id_col, how='left')
)

In [ ]:
final_df.columns
final_df.to_parquet("../output/intermediate/final_df.parquet")

## TableOne

In [ ]:
# Compute the overall table and grab its Variable order
tbl_overall = pyCLIF.make_table_one(final_df, patient_cohort, id_col="encounter_block")
var_order = tbl_overall["Variable"].tolist()         

if pyCLIF.helper["site_name"].lower() == 'mimic':
    # Write table_by_year to CSV in output/final directory
    tbl_overall.to_csv(f"../output/final/table_one_{pyCLIF.helper['site_name']}.csv", index=False)
    # View
    print(tbl_overall.to_markdown(index=False))
else:
    # Build a dict of Series (Overall + each year)
    # get sorted list of all calendar years in your data
    years = sorted(final_df['admission_dttm'].dt.year.unique())
    # build a dict of Series, one for “Overall,” one for each year
    results = {}
    results = {"Overall": tbl_overall.set_index("Variable")["Overall"]}

    for yr in sorted(final_df["admission_dttm"].dt.year.unique()):
        df_y = final_df[final_df["admission_dttm"].dt.year == yr]
        tbl_y = pyCLIF.make_table_one(df_y, patient_cohort, id_col="encounter_block")
        results[str(yr)] = tbl_y.set_index("Variable")["Overall"]

    # 3) Create the wide DataFrame, using the saved var_order
    table_by_year = (
        pd.DataFrame(results)           # index is Variable
        .reindex(var_order)          # enforce your original row order
        .reset_index()               # bring Variable back as a column
        .rename(columns={"index":"Variable"})
    )
    # Write table_by_year to CSV in output/final directory
    table_by_year.to_csv(f"../output/final/table_one_{pyCLIF.helper['site_name']}.csv", index=False)
    # View
    print(table_by_year.to_markdown(index=False))